In [ ]:
' 1. Getting data

'    a.)$wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/$ 

'    b.)then downloading and unzipping next files:

'       GCF_000005845.2_ASM584v2_genomic.fna.gz
'       GCF_000005845.2_ASM584v2_genomic.gff.gz
'       amp_res_1.fastq.zip
'       amp_res_2.fastq.zip

' 2. Inspecting raw sequencing data manually

'    a.)Checking our files with $head -20 [seq_file]$

'    b.)Checking whole fasta with $cat [fasta file]$

'    c.)$wc - l [fasta file]$   
'        1823504 lines
'        number_of_reads = 1823504/4 = 455876

' 3. Inspecting raw sequencing data with fastqc. Filtering the reads.
'    a.) Installing fastqc with $apt-get install fastqc$

'    b.) use $fastqc -o . amp_res_1.fastq amp_res_2.fastq $

'    c.)Basic statistics show Total Sequences = 455876 (math our number_of_reads)

'    d.)Unusual points:
'      amp_res_1_fastqc: Per base sequence quality and Per tile sequence quality

'      amp_res_2_fastqc: Per base sequence quality 

' 4. Filtering the reads 

'  a.) download trimmomatic

'  b.) use $TrimmomaticPE amp_res_1.fastq amp_res_2.fastq amp_res_1.trimmed.fastqc amp_res_1un.trimmed.fastqc amp_res_2.trimmed.fastqc amp_res_2un.trimmed.fastqc LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20  MINLEN:20$

'     Input Read Pairs: 455876 Both Surviving: 446259 (97,89%) Forward Only Surviving: 9216 (2,02%) Reverse Only Surviving: 273 (0,06%) Dropped: 128 (0,03%)

'  c.) $wc -l amp_res_1.trimmed.fastqc$ #1785036 amp_res_1.trimmed.fastqc

'     1785036/4 = 446259

'  d.) use $TrimmomaticPE amp_res_1.fastq amp_res_2.fastq amp_res_30_1.trimmed.fastqc amp_res_30_1un.trimmed.fastqc amp_res_30_2.trimmed.fastqc amp_res_30_2un.trimmed.fastqc LEADING:30 TRAILING:30 SLIDINGWINDOW:10:30  MINLEN:30$

'   Input Read Pairs: 455876 Both Surviving: 360209 (79,01%) Forward Only Surviving: 36716 (8,05%) Reverse Only Surviving: 27291 (5,99%) Dropped: 31660 (6,94%)

'  Q_score = 20 	 Surv_reads = 446259

'  Q_score = 30 	 Surv_reads = 360209

'  e.) repeat fastqc analysis for reads with Q_score = 20 and get only 1 unsusual point Per tile sequence quality for amp_1_trimm reads and 0 for reversed.

' 5. Aligning sequences to reference

'    5.1 Index the reference file 
'       a.) Installing BWA  with $apt-get install bwa$
'       b.) use command $bwa index GCF_000005845.2_ASM584v2_genomic.fna$
'    5.2 Align your reads
'       a.) Use command $bwa mem GCF_000005845.2_ASM584v2_genomic.fna amp_res_1.fastq amp_res_2.fastq > alignment.sam$
'    5.3 Compress SAM file
'       a.) use $samtools view -S -b alignment.sam > alignment.bam$
'       b.) use $samtools flagstat alignment.bam$   (910940 + 0 mapped (99.87% : N/A))
'    5.4 Sort and index BAM file
'       a.) $samtools sort alignment.bam -o alignment_sorted.bam$
'       b.) $samtools index alignment_sorted.bam$
'       c.) downloading IRV browser and visualizing reference genome
' 
' 6. Variant calling
'   
'   a.) Mpileup $samtools mpileup -f GCF_000005845.2_ASM584v2_genomic.fna alignment_sorted.bam >  my.mpileup$
'   b.) downloading VarScan.v2.4.0.jar
'   c.) $java -jar VarScan.v2.4.0.jar  mpileup2snp my.mpileup --min-var-freq N --variants --output-vcf 1 > VarScan_results.vcf$ 
'       N=0.20, also check mutations for N = {0.01, 0.50, 0.99}. Got same results.

' 7. Variant effect prediction

'''''''Exploring mutattions'''''''

'     find 5 mutations in positions:
'NC_000913.3	93043  C 	G  Alanine  	 Glycine (missense ) ftsI gene. protein: peptidoglycan DD-transpeptidase FtsI
'NC_000913.3	482698 T	A  Glutamine 	 Leucine (missense ) acrB gene. protein: multidrug efflux pump RND permease AcrB
'NC_000913.3	852762 A	G  not coding region
'NC_000913.3	3535147 A	C  stop 	Glutamic acid (nonsense) envZ gene. protein: sensory histidine kinase EnvZ
'NC_000913.3	4390754 G	T  Alanine  	 Alanine (synonymous)

